In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\aleem\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Load dataset from capiq. These are pharma/ biotech companies part of nasdaq/ s&p500, Dow Jones and other US exchanges
capiq_biotech = pd.read_excel("CapIQ_biotech_screening.xlsx")
capiq_biotech.head()

,Company Name,Ticker,Security Name,Trading Status,"Most Recent Trade Price ($USD, Historical rate)",Equity Security Type,Industry Classifications,Company Status,Index Constituents
0,CorMedix Inc.,AMEX:CRMD,Class A Common Stock,Active,5.79,Common Stock,Pharmaceuticals (Primary),Operating,Russell 3000 Index
1,CEL-SCI Corporation,AMEX:CVM,Common Stock,Active,12.24,Common Stock,Biotechnology (Primary),Operating,Russell 3000 Index
2,"iBio, Inc.",AMEX:IBIO,Common Stock,Active,2.05,Common Stock,Biotechnology (Primary),Operating,Russell 3000 Index
3,OncoCyte Corporation,AMEX:OCX,Common Shares,Active,1.79,Common Stock,Biotechnology (Primary),Operating,Russell 3000 Index
4,Abeona Therapeutics Inc.,NasdaqCM:ABEO,Common Stock,Active,1.29,Common Stock,Biotechnology (Primary),Operating,NASDAQ Composite Index; Russell 3000 Index


In [3]:
# This dataset is from the website: ClinicalTrials.gov. This contains phase 2 and 3 studies from the US on Oncology
oncology_database = pd.read_csv("Oncology_Dataset.csv")
oncology_database.head(3)

,Rank,Sponsor/Collaborators,Title,Status,Study Results,Conditions,Interventions,Funded Bys,Study Type,Locations,URL
0,1,"University of Wisconsin, Madison|Dendreon",Provenge With or Without pTVG-HP DNA Booster V...,"Active, not recruiting",Has Results,Prostate Cancer,Biological: sipuleucel-T|Biological: DNA Vaccine,Other|Industry,Interventional,"University of Wisconsin Carbone Cancer Center,...",https://ClinicalTrials.gov/show/NCT01706458
1,2,Cedars-Sinai Medical Center|Merck Sharp & Dohm...,A Study of Pembrolizumab With Standard Treatme...,"Active, not recruiting",Has Results,Ovarian Cancer,Drug: Pembrolizumab|Drug: Gemcitabine|Drug: Ci...,Other|Industry,Interventional,"Cedars-Sinai Medical Center, Los Angeles, Cali...",https://ClinicalTrials.gov/show/NCT02608684
2,3,University of Kansas Medical Center|AstraZeneca,Anti-Hormone Therapy (With Anastrazole and Ful...,"Active, not recruiting",Has Results,Invasive Breast Cancer,Drug: Fulvestrant|Drug: Anastrazole,Other|Industry,Interventional,"University of Kansas Medical Center, Kansas Ci...",https://ClinicalTrials.gov/show/NCT00921115


In [4]:
# Run the matching algorithm. For each entry from the oncology dataset, the algorithm goes through each entry
# in the capiq dataset to see if there is a potential match in the name of the companies. The higher the score,
# the better the match. 
matched_vendors = []

for row in oncology_database.index:
    sponsor_name = oncology_database._get_value(row,"Sponsor/Collaborators")
    for columns in capiq_biotech.index:
        capiq_company = capiq_biotech._get_value(columns,"Company Name")
        matched_token=fuzz.partial_ratio(sponsor_name,capiq_company)
        matched_vendors.append([sponsor_name,capiq_company,matched_token])
            
matched_vendors = pd.DataFrame(matched_vendors)
matched_vendors.columns = ["Sponsors/ Collaborator", "CapIQ company", "Score"]
matched_vendors.head()

,Sponsors/ Collaborator,CapIQ company,Score
0,"University of Wisconsin, Madison|Dendreon",CorMedix Inc.,31
1,"University of Wisconsin, Madison|Dendreon",CEL-SCI Corporation,26
2,"University of Wisconsin, Madison|Dendreon","iBio, Inc.",30
3,"University of Wisconsin, Madison|Dendreon",OncoCyte Corporation,35
4,"University of Wisconsin, Madison|Dendreon",Abeona Therapeutics Inc.,21


In [5]:
# Check results by sorting it by the score
pd.set_option('display.max_colwidth', -1)
matched_vendors.sort_values(by=['Score'], ascending = False).head(30)

C:\Users\aleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Sponsors/ Collaborator,CapIQ company,Score
31000,Eli Lilly and Company,Eli Lilly and Company,100
81896,Incyte Corporation,Incyte Corporation,100
84612,Bristol-Myers Squibb,Bristol-Myers Squibb Company,100
59428,Pfizer,Pfizer Inc.,100
183183,"Verastem, Inc.","Verastem, Inc.",100
168639,"AVEO Pharmaceuticals, Inc.","AVEO Pharmaceuticals, Inc.",100
65022,AstraZeneca,AstraZeneca PLC,100
76650,AstraZeneca,AstraZeneca PLC,100
80053,Sanofi,Sanofi,100
21818,Incyte Corporation|Merck Sharp & Dohme Corp.,Incyte Corporation,100


In [6]:
# We want to find the max score for each entry in "Sponsors/ Collaborator" column by grouping the values together.
# As you can see, there are multiple values for "University of Wisconsin, Madison|Dendreon" as the algo finds a score for 
# this for each entry in the CapIQ column. Find the entry with the max score because that gives us the most similar value

idx = matched_vendors.groupby(['Sponsors/ Collaborator'])['Score'].transform(max) == matched_vendors['Score']
df = matched_vendors[idx]
df = pd.DataFrame(df)
df.head(30)

,Sponsors/ Collaborator,CapIQ company,Score
595,"University of Wisconsin, Madison|Dendreon",Sanofi,50
892,Cedars-Sinai Medical Center|Merck Sharp & Dohme Corp.,"Clearside Biomedical, Inc.",58
1714,University of Kansas Medical Center|AstraZeneca,AstraZeneca PLC,85
2121,H. Lee Moffitt Cancer Center and Research Institute|Merck Sharp & Dohme Corp.|Prometheus Laboratories,"SenesTech, Inc.",60
3216,H. Lee Moffitt Cancer Center and Research Institute|Bristol-Myers Squibb,Bristol-Myers Squibb Company,83
3413,"H. Lee Moffitt Cancer Center and Research Institute|Genentech, Inc.","SenesTech, Inc.",80
4155,Duke University|Janssen Pharmaceuticals,GW Pharmaceuticals plc,80
4705,H. Lee Moffitt Cancer Center and Research Institute|Bayer,"SenesTech, Inc.",60
5800,H. Lee Moffitt Cancer Center and Research Institute|Bristol-Myers Squibb,Bristol-Myers Squibb Company,83
6446,Dana-Farber Cancer Institute|Bristol-Myers Squibb,Bristol-Myers Squibb Company,83


In [7]:
# Join the above dataframe with the oncology database on the "Sponsors/Collaborator" column. We are doing this as 
# we would want all the details for each study along with the respective matching scores
df1 = df.join(oncology_database.set_index('Sponsor/Collaborators'), on='Sponsors/ Collaborator')

# Now join df1 with the capiq_biotech dataset so that we can have the ticker symbols as well
df2 = df1.join(capiq_biotech[['Company Name', 'Ticker']].set_index('Company Name'), on = "CapIQ company")
df2.head()

,Sponsors/ Collaborator,CapIQ company,Score,Rank,Title,Status,Study Results,Conditions,Interventions,Funded Bys,Study Type,Locations,URL,Ticker
595,"University of Wisconsin, Madison|Dendreon",Sanofi,50,1,Provenge With or Without pTVG-HP DNA Booster Vaccine in Prostate Cancer,"Active, not recruiting",Has Results,Prostate Cancer,Biological: sipuleucel-T|Biological: DNA Vaccine,Other|Industry,Interventional,"University of Wisconsin Carbone Cancer Center, Madison, Wisconsin, United States",https://ClinicalTrials.gov/show/NCT01706458,NasdaqGS:SNY
892,Cedars-Sinai Medical Center|Merck Sharp & Dohme Corp.,"Clearside Biomedical, Inc.",58,2,A Study of Pembrolizumab With Standard Treatment in Patients With Recurrent Platinum-resistant Ovarian Cancer,"Active, not recruiting",Has Results,Ovarian Cancer,Drug: Pembrolizumab|Drug: Gemcitabine|Drug: Cisplatin,Other|Industry,Interventional,"Cedars-Sinai Medical Center, Los Angeles, California, United States",https://ClinicalTrials.gov/show/NCT02608684,NasdaqGM:CLSD
1714,University of Kansas Medical Center|AstraZeneca,AstraZeneca PLC,85,3,Anti-Hormone Therapy (With Anastrazole and Fulvestrant) Before Surgery to Treat Postmenopausal Women With Breast Cancer.,"Active, not recruiting",Has Results,Invasive Breast Cancer,Drug: Fulvestrant|Drug: Anastrazole,Other|Industry,Interventional,"University of Kansas Medical Center, Kansas City, Kansas, United States",https://ClinicalTrials.gov/show/NCT00921115,NasdaqGS:AZN
2121,H. Lee Moffitt Cancer Center and Research Institute|Merck Sharp & Dohme Corp.|Prometheus Laboratories,"SenesTech, Inc.",60,4,Interleukin-2 and Pembrolizumab for Metastatic Kidney Cancer,"Active, not recruiting",Has Results,Kidney Cancer,Drug: Pembrolizumab|Drug: Interleukin-2,Other|Industry,Interventional,"H. Lee Moffitt Cancer Center and Research Institute, Tampa, Florida, United States",https://ClinicalTrials.gov/show/NCT02964078,NasdaqCM:SNES
3216,H. Lee Moffitt Cancer Center and Research Institute|Bristol-Myers Squibb,Bristol-Myers Squibb Company,83,5,Study of Nivolumab in Patients With Advanced Refractory Biliary Tract Cancers,"Active, not recruiting",Has Results,Biliary Tract Cancer|Biliary Tract Neoplasms,Drug: Nivolumab,Other|Industry,Interventional,"City of Hope Cancer Center, Duarte, California, United States|H. Lee Moffitt Cancer Center and Research Institute, Tampa, Florida, United States|Emory University Winship Cancer Institute, Atlanta, Georgia, United States",https://ClinicalTrials.gov/show/NCT02829918,NYSE:BMY


In [8]:
# The ticker symbol column is the last column in the above dataframe after joining the 2 dataframes. We want to rearrange
# the columns so that the ticker column is next to the company name
cols = df2.columns.tolist()
rearrange_cols = cols[0:2] + cols[-1:] + cols[2:-1]
rearrange_cols

['Sponsors/ Collaborator',
 'CapIQ company',
 'Ticker',
 'Score',
 'Rank',
 'Title',
 'Status',
 'Study Results',
 'Conditions',
 'Interventions',
 'Funded Bys',
 'Study Type',
 'Locations',
 'URL']

In [9]:
# This gives us the new dataframe with the reaaranged columns
df3 = df2[rearrange_cols]
df3

# We only want to look at scores that are above 93. Scores below 93 are not good for matching purposes.
cleaned_dataset = df3[df3.Score > 93]

# Remove duplicates which are a result of joining the dataframes
cleaned_dataset_final = cleaned_dataset.drop_duplicates()
cleaned_dataset_final.head(4)

Sponsors/ Collaborator  \
8780   Andrew Sikora|Advaxis, Inc.|Baylor College of Medicine               
12774  Georgetown University|Merck Sharp & Dohme Corp.|Incyte Corporation   
14850  Eli Lilly and Company                                                
14850  Eli Lilly and Company                                                

               CapIQ company         Ticker  Score  Rank  \
8780   Advaxis, Inc.          NasdaqGS:ADXS  100    14     
12774  Incyte Corporation     NasdaqGS:INCY  100    20     
14850  Eli Lilly and Company  NYSE:LLY       100    23     
14850  Eli Lilly and Company  NYSE:LLY       100    48     

                                                                                     Title  \
8780   ADXS 11-001 Vaccination Prior to Robotic Surgery, HPV-Positive Oropharyngeal Cancer   
12774  Pembrolizumab and Epacadostat in Patients With Thymic Carcinoma                       
14850  A Study of Ramucirumab (IMC-1121B) and Paclitaxel in Participants With Solid Tumors   
14850  Phase III Study of Docetaxel + Ramucirumab or Placebo in Breast Cancer                

                       Status Study Results  \
8780   Active, not recruiting  Has Results    
12774  Active, not recruiting  Has Results    
14850  Active, not recruiting  Has Results    
14850  Active, not recruiting  Has Results    

                                                                                                                 Conditions  \
8780   Head and Neck Cancer|Squamous Cell Carcinoma of the Head and Neck|HPV Positive Oropharyngeal Squamous Cell Carcinoma   
12774  Thymic Carcinoma|Thymus Neoplasms|Thymus Cancer                                                                        
14850  Malignant Solid Tumor                                                                                                  
14850  Breast Cancer                                                                                                          

                                                            Interventions  \
8780   Biological: ADXS11-001 (ADXS-HPV)                                    
12774  Drug: Pembrolizumab|Drug: Epacadostat                                
14850  Biological: ramucirumab (IMC-1121B)|Drug: paclitaxel                 
14850  Biological: ramucirumab (IMC-1121B)|Drug: docetaxel|Other: Placebo   

           Funded Bys      Study Type  \
8780   Other|Industry  Interventional   
12774  Other|Industry  Interventional   
14850  Industry        Interventional   
14850  Industry        Interventional   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     